In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import bernoulli
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [2]:
mushrooms = pd.read_csv('mushroom.csv')

### I.   $\phi(x) = x$

In [3]:
class_shrooms = mushrooms['class']
mushrooms = mushrooms.drop(['class'], axis=1)
list_of_cols = mushrooms.columns
mushrooms = StandardScaler().fit_transform(X=mushrooms)

In [4]:
mushrooms = pd.DataFrame(mushrooms)
mushrooms.columns = list_of_cols
mushrooms['class'] = class_shrooms

In [5]:
n_samples = mushrooms.shape[0]
mushrooms['label'] = np.ones(n_samples)*(-1)
for i in range(n_samples):
    random = bernoulli.rvs(p=3/4)
    if mushrooms.loc[i,'class'] == 1 and random == 0:
        mushrooms.loc[i,'label'] = 1
    else:
        mushrooms.loc[i,'label'] = -1
mushrooms.groupby('label').count()

,cap-diameter,cap-shape,gill-attachment,gill-color,stem-height,stem-width,stem-color,season,class
label,,,,,,,,,
-1.0,46747,46747,46747,46747,46747,46747,46747,46747,46747
1.0,7288,7288,7288,7288,7288,7288,7288,7288,7288


In [6]:
n_cluster = 8
clusterized_data = KMeans(n_clusters=n_cluster, init='random').fit(mushrooms.drop(['label', 'class'], axis=1).to_numpy())
mushrooms['cluster'] = clusterized_data.labels_
mushrooms[['class','label','cluster']].groupby('cluster').sum()

,class,label
cluster,,
0,5990,-5621.0
1,3300,-5571.0
2,3752,-4067.0
3,2660,-3314.0
4,5017,-7120.0
5,2362,-5089.0
6,5371,-5269.0
7,1223,-3408.0


In [7]:
list_of_ratio = []
for i in range(n_cluster):
    list_of_ratio.append(mushrooms[mushrooms['cluster'] == i]['label'].sum()/mushrooms[mushrooms['cluster'] == i]['class'].shape[0])
list_of_ratio = np.array(list_of_ratio)
positive_cluster = np.argmax(list_of_ratio)
negative_cluster = np.argmin(list_of_ratio)
print('the cluster containing positive values is : ', positive_cluster, '\n', 
      'the one containing negative exampels is : ', negative_cluster)


the cluster containing positive values is :  0 
 the one containing negative exampels is :  7


In [8]:
reliable_positives = mushrooms[mushrooms['cluster'] == positive_cluster]
reliable_positives = reliable_positives[reliable_positives['label'] == 1]
reliable_negatives = mushrooms[mushrooms['cluster'] == negative_cluster]
reliable_negatives = reliable_negatives[reliable_negatives['label'] == -1]

In [9]:
print(reliable_positives.shape[0], reliable_negatives.shape[0])

1446 3743


In [10]:
reliable_negatives = reliable_negatives.sample(n=reliable_positives.shape[0])

In [11]:
#we suppose (it is true but for the test) that the ratio of correctly labeled example 
#is 1/2